In [ ]:
import flopy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sim_name = "zaidel"

# Model units
length_units = "meters"
time_units = "days"

# Model parameters
nper = 1  # Number of periods
nlay = 1  # Number of layers
nrow = 1  # Number of rows
ncol = 100  # Number of columns
delr = 5.0  # Column width
delc = 1.0  # Row width
top = np.zeros(ncol, dtype=float) + 25.0  # Top of the model
strt = 23.0  # Starting head
icelltype = 1  # Cell conversion type
k11 = 0.0001  # Horizontal hydraulic conductivity

H1 = 23.0  # Constant head in column 1
H2 = 2.0 # Constant head in column <ncol>

# Time discretization
tdis_ds = ((1.0, 1, 1.0),)

# Initial bottom at zero
botm = np.zeros((nlay, nrow, ncol), dtype=float)

# Solver parameters
nouter = 100
ninner = 100
hclose = 1e-9
rclose = 1e-6

# stress period data for CHD
chd_spd = [
        [0, 0, 0, H1],
        [0, 0, ncol - 1, H2],
    ]


In [ ]:
# option: add staircase bottom profile or valley (not both)
staircase = False
valley = True

if staircase:
    nsteps = 5
    for j in range(ncol):
        botm[0, :, j] = 20.0 - 5.0 * (nsteps*j//ncol)

# dips the top level with a fraction to make a valley
if valley:
    dip_fraction = 0.5    
    for j in range(ncol):
        top[j] = top[j] * dip_fraction *(abs(j - ncol//2)/(ncol//2)) + top[j] * (1.0 - dip_fraction) 


In [ ]:
# add drain on top
drain_data = []
for j in range(ncol):
    elev = top[j]
    cond_drain = 100.0
    drain_data.append([(0, 0, j), elev, cond_drain])

In [ ]:
# setup simulation
sim_ws = sim_name
sim = flopy.mf6.MFSimulation(sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6")
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units)
ims = flopy.mf6.ModflowIms(
    sim,
    # under_relaxation="dbd",
    # under_relaxation_theta=0.8,
    # under_relaxation_kappa=0.0001,
    # under_relaxation_gamma=0.0,
    # under_relaxation_momentum=0.0,
    # backtracking_number=20,
    # backtracking_tolerance=1.05,
    # backtracking_reduction_factor=0.1,
    # backtracking_residual_limit=0.002,
    print_option="all",
    linear_acceleration="bicgstab",
    outer_maximum=nouter,
    outer_dvclose=hclose,
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=f"{rclose}",
)
gwf = flopy.mf6.ModflowGwf(sim, modelname=sim_name, newtonoptions="newton")
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    icelltype=icelltype,
    k=k11,
)
ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)

drn = flopy.mf6.ModflowGwfdrn(gwf, stress_period_data=drain_data)

head_filerecord = f"{sim_name}.hds"
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=head_filerecord,
    saverecord=[("HEAD", "ALL")],
)

sim.write_simulation()

In [ ]:
# run
sim.run_simulation()

In [ ]:
# plot
fig = plt.figure(figsize=(12,3))
pxs = flopy.plot.PlotCrossSection(model=gwf, line={"row": 0})

head = gwf.output.head().get_data()
pa = pxs.plot_array(head, head=head)
pxs.plot_grid()
pxs.plot_bc("CHD")

plt.colorbar(pa)
